In [10]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np
import matplotlib.cm as cm
from matplotlib.colors import LogNorm, Normalize
from matplotlib.patches import Wedge
import h5py
import torch

In [11]:
# plotting functions for the cylinder/calorimeter: (made by Hao and copied into this notebook)

def plot_hist(f, fn):
    fig, axes = plt.subplots(1,3, figsize=(15, 3), sharey=False, sharex=False, tight_layout=False)
    fig.text(0.5, 1.01, f'{fn}', ha='center', fontsize=15)
    axes[0].hist(f['eta_mod'], bins=50, color='skyblue')
    axes[0].set_xlabel('eta_mod')
    axes[0].set_ylabel('Histogram')
    # plt.show()
    axes[1].hist(f['phi_mod'], bins=50, color='magenta')
    axes[1].set_xlabel('phi_mod')
    axes[1].set_ylabel('Histogram')
    # plt.show()
    axes[2].hist(np.log2(f['incident_energy']), density=False, log=True, bins=np.arange(8, 24, 0.5))
    axes[2].set_xlabel('log_2(E)')
    axes[2].set_ylabel('Histogram')
    plt.show()

def add_cylindrical_voxel(ax, r_in, r_out, theta0, theta1, z_low, z_high, num_points=20, color=None):
    """
    Adds a single cylindrical voxel (wedge) to the 3D axis.
    
    Parameters:
      ax       : Matplotlib 3D axis.
      r_in     : Inner radius.
      r_out    : Outer radius.
      theta0   : Starting angle in radians.
      theta1   : Ending angle in radians.
      z_low    : Lower z-bound.
      z_high   : Upper z-bound.
      num_points: Number of points along the arc.
      color    : Color to fill the voxel.
    """
    
    # Generate a smooth arc between theta0 and theta1.
    theta = np.linspace(theta0, theta1, num_points)
    
    # Top face arcs
    outer_top = np.column_stack((r_out * np.cos(theta), r_out * np.sin(theta), np.full(theta.shape, z_high)))
    inner_top = np.column_stack((r_in * np.cos(theta), r_in * np.sin(theta), np.full(theta.shape, z_high)))
    
    # Bottom face arcs
    outer_bot = np.column_stack((r_out * np.cos(theta), r_out * np.sin(theta), np.full(theta.shape, z_low)))
    inner_bot = np.column_stack((r_in * np.cos(theta), r_in * np.sin(theta), np.full(theta.shape, z_low)))
    
    # Build top and bottom faces (by joining outer and inner arcs)
    top_face = np.concatenate((outer_top, inner_top[::-1]))
    bottom_face = np.concatenate((outer_bot, inner_bot[::-1]))
    
    # Create side faces at the angular boundaries (using first and last points)
    side_face_start = np.array([outer_bot[0], outer_top[0], inner_top[0], inner_bot[0]])
    side_face_end   = np.array([outer_bot[-1], outer_top[-1], inner_top[-1], inner_bot[-1]])
    
    faces = [top_face, bottom_face, side_face_start, side_face_end]
    
    poly3d = Poly3DCollection(faces, alpha=0.6, edgecolor='k', facecolor=color)
    ax.add_collection3d(poly3d)

def add_layer_varying(ax, radii, thetas, z_low, z_high, num_points=20, color=None):
    """
    Constructs one layer of the cylinder when the angular partitions vary by ring.
    
    The radii and theta arrays are 1D with repeated values. We group them
    and, for each annular cell between unique radii[i] and unique radii[i+1],
    we use the theta boundaries from the outer ring (group i+1). This function
    also adds a closing wedge to complete the circle.
    
    Parameters:
      ax     : The matplotlib 3D axis.
      radii  : 1D array of radii boundaries (with repeats).
      thetas : 1D array of theta boundaries (with repeats).
      z_low  : Lower z-bound for this layer.
      z_high : Upper z-bound for this layer.
      color  : Color for this layer.
    """
    unique_radii, r_counts = np.unique(radii, return_counts=True)
    
    # Split the theta array into groups using the counts from radii.
    theta_groups = []
    idx = 0
    for count in r_counts:
        theta_groups.append(thetas[idx:idx+count])
        idx += count

    # Loop over annular cells.
    for i in range(len(unique_radii) - 1):
        r_in = unique_radii[i]
        r_out = unique_radii[i+1]
        # Choose theta boundaries from the outer ring (group i+1).
        if i+1 < len(theta_groups):
            theta_boundaries = theta_groups[i+1]
        else:
            theta_boundaries = theta_groups[-1]
            
        # Create wedges for each angular segment.
        for j in range(len(theta_boundaries) - 1):
            theta0 = theta_boundaries[j]
            theta1 = theta_boundaries[j+1]
            add_cylindrical_voxel(ax, r_in, r_out, theta0, theta1, z_low, z_high, num_points, color)
        
        # --- Add the closing wedge ---
        theta0 = theta_boundaries[-1]
        theta1 = theta_boundaries[0] + 2 * np.pi
        add_cylindrical_voxel(ax, r_in, r_out, theta0, theta1, z_low, z_high, num_points, color)

def draw_cylinder(f):
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')

    num_layers = 23
    layer_thickness = 1.0  # Adjust vertical spacing as needed

    colormap = cm.get_cmap("viridis")

    for layer in range(num_layers):
        z_low = layer * layer_thickness
        z_high = (layer + 1) * layer_thickness
        # Get a color from the colormap based on the layer index.
        color = colormap(layer / num_layers)
        add_layer_varying(ax, f[f'binstart_radius_layer_{layer}'], f[f'binstart_alpha_layer_{layer}'], z_low, z_high, num_points=20, color=color)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_xlim(-1200, 1200)
    ax.set_ylim(-1200, 1200)
    ax.set_zlim(0, num_layers * layer_thickness)
    plt.title("Cylinder with 23 Layers (Different Color per Layer)")
    plt.show()

class PLT_ATLAS:
    def __init__(self, raw_dir, event_n, all_layers=None):
        """
        Initialize the PLT_ATLAS object.

        Parameters:
        - raw_dir: Path to the raw HDF5 data file.
        - event_n: Event number to process.
        - all_layers: List of layer numbers to plot. Defaults to [0, 1, 2, 3, 12].
        """
        if all_layers is None:
            all_layers = [i for i in range(24)]#[0, 1, 2, 3, 12]
        self.ATLAS_raw_dir = raw_dir
        self.load_dataset()
        self.event_n = event_n
        self.all_layers = all_layers
        
    def load_dataset(self):
        with h5py.File(self.ATLAS_raw_dir, 'r') as file:
            # List all groups
            self.data = {}
            #print("Keys: %s" % list(file.keys()))
            for key in file.keys():
                self.data[key] = torch.tensor(np.array(file[key]))

    def get_section_info(self):
        """
        Process and retrieve sector information for plotting.

        Returns:
        - List of tuples containing (inner_r, outer_r, start_angle, end_angle, energy).
        """
        # Convert angles from radians to degrees and round
        binstart_alpha_deg = torch.round(self.binstart_alpha_info / torch.pi * 180)
        binend_alpha_deg = binstart_alpha_deg + torch.round(self.binsize_alpha_info / torch.pi * 180)
        
        # Calculate end radius
        binend_radius_info = self.binstart_radius_info + self.binsize_radius_info
        
        # Combine all sector information into a list of tuples
        sectors = list(zip(
            self.binstart_radius_info.tolist(),
            binend_radius_info.tolist(),
            binstart_alpha_deg.tolist(),
            binend_alpha_deg.tolist(),
            self.single_event_energy.tolist()
        ))
        return sectors

    def plot_calorimeter(self, ax, scale='equal_bin', cmap=cm.rainbow, norm=None, title=None):
        """
        Plot the Calorimeter Layer Energy Diagram on a given axis.

        Parameters:
        - ax: Matplotlib axis to plot on.
        - scale: 'linear' or 'equal_bin', determines the type of radial scale.
        - cmap: Colormap for the energy representation.
        - norm: Normalization for the colormap.
        - title: Title for the subplot.
        """
        sectors = self.sectors

        # If normalization is not provided, create it based on current energies
        if norm is None:
            energies = [sector[4] for sector in sectors]
            norm = LogNorm(vmin=max(min(values), 0.0001), vmax=max(max(values), 0.0001))

        if scale == 'linear':
            # Linear radial scale
            max_radius = max(sector[1] for sector in sectors)
            transform = lambda r: r
        elif scale == 'equal_bin':
            # Equal bin radial scale
            radial_boundaries = sorted({r for sector in sectors for r in (sector[0], sector[1])})
            num_layers = len(radial_boundaries) - 1
            plot_radial_boundaries = np.linspace(0, 1, num_layers + 1)
            

            def transform(r):
                """
                Map actual radius to plot radius ensuring equal space for each layer.
                """
                for i in range(num_layers):
                    if radial_boundaries[i] <= r < radial_boundaries[i + 1]:
                        layer_fraction = (r - radial_boundaries[i]) / (radial_boundaries[i + 1] - radial_boundaries[i])
                        return plot_radial_boundaries[i] + layer_fraction * (plot_radial_boundaries[i + 1] - plot_radial_boundaries[i])
                return plot_radial_boundaries[-1]  # Handle the maximum boundary

            max_radius = plot_radial_boundaries[-1]
        else:
            raise ValueError("The 'scale' parameter must be 'linear' or 'equal_bin'")

        # Plot each sector as a wedge
        for inner_r, outer_r, start_angle, end_angle, energy in sectors:
            color = cmap(norm(energy))
            if scale == 'equal_bin':
                transformed_inner = transform(inner_r)
                transformed_outer = transform(outer_r)
                width = transformed_outer - transformed_inner
                wedge = Wedge(
                    center=(0, 0),
                    r=transformed_outer,
                    theta1=start_angle,
                    theta2=end_angle,
                    width=width,
                    edgecolor='black',
                    facecolor=color,
                    alpha=1
                )
            else:  # 'linear'
                wedge = Wedge(
                    center=(0, 0),
                    r=transform(outer_r),
                    theta1=start_angle,
                    theta2=end_angle,
                    width=transform(outer_r) - transform(inner_r),
                    edgecolor='black',
                    facecolor=color,
                    alpha=1
                )
            try:
                ax.add_patch(wedge)
            except:
                pass

        # Configure plot limits and appearance
        ax.set_xlim(-max_radius - 0.1, max_radius + 0.1)
        ax.set_ylim(-max_radius - 0.1, max_radius + 0.1)
        ax.set_aspect('equal')
        ax.axis('off')

        # Add title if provided
        if title:
            #print(title)
            ax.set_title(title, fontsize=15)

    def plot_all_layers(self, event_n, scale='equal_bin', vmin=1e-2, vmax=1e4):
        """
        Plot all specified layers of the calorimeter for the given event in a composite figure.

        Parameters:
        - scale: 'linear' or 'equal_bin', determines the type of radial scale for all subplots.
        """
        self.event_n = event_n
        num_layers = len(self.all_layers)
        # Create a figure with subplots arranged in one row
        figsize = (15, 15)
        fig = plt.figure(figsize=figsize, dpi=200)
        # fig, axes = plt.subplots(4, 5, figsize=(6 * num_layers, 7), constrained_layout=True) # figsize=(6 * num_layers, 7)
        
        
        # Ensure axes is iterable
        if num_layers == 1:
            axes = [axes]

        # Initialize variables to determine global normalization across all layers
        all_energies = []

        # First pass: Collect all energies to set a common normalization

        # Define global normalization
        norm = LogNorm(vmin=vmin, vmax=vmax) # changed from incident_energy
        incident_energy = torch.tensor(self.data["incident_energy"][self.event_n])
        print(f"event info: event_n = {self.event_n}, incident_energy = {incident_energy.item()}")

        # Iterate through each layer and plot on the corresponding axis
        for i, layer in enumerate(self.all_layers):
            # Retrieve and process data for the current layer
            # energy_info = torch.tensor(self.data[f'energy_layer_{layer}'][self.event_n, :])
            # binsize_alpha = torch.tensor(self.data[f"binsize_alpha_layer_{layer}"][:])
            # binstart_alpha = torch.tensor(self.data[f"binstart_alpha_layer_{layer}"][:])
            # binsize_radius = torch.tensor(self.data[f"binsize_radius_layer_{layer}"][:])
            # binstart_radius = torch.tensor(self.data[f"binstart_radius_layer_{layer}"][:])
            energy_info = self.data[f'energy_layer_{layer}'][self.event_n, :]
            binsize_alpha = self.data[f"binsize_alpha_layer_{layer}"]
            binstart_alpha = self.data[f"binstart_alpha_layer_{layer}"]
            binsize_radius = self.data[f"binsize_radius_layer_{layer}"]
            binstart_radius = self.data[f"binstart_radius_layer_{layer}"]
            single_event_energy = energy_info * incident_energy

            # Assign data to instance variables for section info retrieval
            self.binsize_alpha_info = binsize_alpha
            self.binstart_alpha_info = binstart_alpha
            self.binsize_radius_info = binsize_radius
            self.binstart_radius_info = binstart_radius
            self.single_event_energy = single_event_energy

            # Get sector information
            self.sectors = self.get_section_info()

            # Plot on the provided axis with a common normalization
            ax = plt.subplot(5, 5, i+1)
            self.plot_calorimeter(ax=ax, scale=scale, norm=norm, title=f'Layer {layer}')

        # Add a single colorbar for all subplots
        # Positioning the colorbar to the right of all subplots
        fig.suptitle(f'Calorimeter Layer Energy Diagram when E = {incident_energy/1000:.2f} GeV\n', fontsize=15)
        cbar_ax = fig.add_axes([0.05, -0.1, 0.9, 0.1])  # [left, bottom, width, height]
        cmap = 'rainbow' #cm.viridis
        sm = cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = fig.colorbar(sm, cax=cbar_ax, location='bottom', orientation='horizontal')
        cbar.set_label('Energy in MeV', fontsize=15)
        cbar.ax.tick_params(labelsize=20)

        plt.show()

In [15]:
def plot_layer_geometry_from_h5(h5file, layer, scale='equal_bin', ax=None, title=None,
                                edgecolor='k', linewidth=0.4):
    """
    function to plot the layers while showing the wedges
    will also print out layer geometry
    """
    # load binning arrays (geometry)
    r0 = np.asarray(h5file[f'binstart_radius_layer_{layer}'])
    dr = np.asarray(h5file[f'binsize_radius_layer_{layer}'])
    a0 = np.asarray(h5file[f'binstart_alpha_layer_{layer}'])   # radians
    da = np.asarray(h5file[f'binsize_alpha_layer_{layer}'])    # radians

    r1  = r0 + dr
    a1  = a0 + da
    a0d = np.rad2deg(a0)  # degrees for plotting
    a1d = np.rad2deg(a1)

    # equal-bin display mapping (can also use 'linear')
    if scale == 'equal_bin':
        bounds = np.unique(np.concatenate([r0, r1]))
        nb = len(bounds) - 1
        plot_bounds = np.linspace(0.0, 1.0, nb+1)
        def xform(R):
            idx = np.searchsorted(bounds, R, side='right') - 1
            idx = np.clip(idx, 0, nb-1)
            frac = (R - bounds[idx]) / (bounds[idx+1] - bounds[idx] + 1e-12)
            return plot_bounds[idx] + frac*(plot_bounds[idx+1] - plot_bounds[idx])
        r0p, r1p = xform(r0), xform(r1)
        Rmax_plot = 1.0
    else:
        r0p, r1p = r0, r1
        Rmax_plot = r1.max() if r1.size else 1.0

    # physical XY values to print
    xs = np.concatenate([r1*np.cos(a0), r1*np.cos(a1), r0*np.cos(a0), r0*np.cos(a1)])
    ys = np.concatenate([r1*np.sin(a0), r1*np.sin(a1), r0*np.sin(a0), r0*np.sin(a1)])
    min_x, max_x = xs.min(initial=0.0), xs.max(initial=0.0)
    min_y, max_y = ys.min(initial=0.0), ys.max(initial=0.0)

    # plot outlines
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 6))
    for inner, outer, th0, th1 in zip(r0p, r1p, a0d, a1d):
        ax.add_patch(Wedge((0, 0), outer, th0, th1,
                           width=outer-inner, facecolor='none',
                           edgecolor=edgecolor, linewidth=linewidth))
    ax.set_xlim(-Rmax_plot-0.05, Rmax_plot+0.05)
    ax.set_ylim(-Rmax_plot-0.05, Rmax_plot+0.05)
    ax.set_aspect('equal'); ax.axis('off')
    ax.set_title(title or f'Layer {layer}', fontsize=12)

    return ax, (len(r0), min_x, max_x, min_y, max_y)


def plot_all_layers_geometry_grid(h5_path, layers=None, scale='equal_bin', cols=6):
    """
    function to plot the layer geometry as a grid
    """
    with h5py.File(h5_path, "r") as f:
        # get layers
        if layers is None:
            layers = sorted({
                int(k.split('_')[-1])
                for k in f.keys()
                if k.startswith('binstart_radius_layer_')
            })

        # making figure
        rows = int(np.ceil(len(layers) / cols))
        fig, axes = plt.subplots(rows, cols, figsize=(cols*3, rows*3), subplot_kw={'aspect':'equal'})
        axes = axes.flatten()

        # cross check:
        total_vox_geom = 0
        total_vox_energy = 0
        global_xmin, global_xmax = np.inf, -np.inf
        global_ymin, global_ymax = np.inf, -np.inf

        for ax, L in zip(axes, layers):
            # geometry plot + count
            ax, (vox_geom, xmin, xmax, ymin, ymax) = plot_layer_geometry_from_h5(
                f, layer=L, scale=scale, ax=ax, title=f'Layer {L}'
            )
            total_vox_geom += vox_geom
            global_xmin = min(global_xmin, xmin); global_xmax = max(global_xmax, xmax)
            global_ymin = min(global_ymin, ymin); global_ymax = max(global_ymax, ymax)

            # dataset cross-check: energy_layer_{L} second dimension is voxel count
            if f.get(f'energy_layer_{L}') is not None:
                vox_energy = f[f'energy_layer_{L}'].shape[1]
                total_vox_energy += vox_energy
                ok = (vox_geom == vox_energy)
                print(f"[Layer {L:2d}] voxels geom={vox_geom:5d} | energy shape={vox_energy:5d} | match={ok} "
                      f"| x vals:[{xmin:8.3f}, {xmax:8.3f}], y vals:[{ymin:8.3f}, {ymax:8.3f}]")
            else:
                print(f"[Layer {L:2d}] voxels geom={vox_geom:5d} | energy shape=N/A  "
                      f"| x vals:[{xmin:8.3f}, {xmax:8.3f}], y vals:[{ymin:8.3f}, {ymax:8.3f}]")

        # get rid of unused subplots
        for ax in axes[len(layers):]:
            ax.axis('off')

        # overall summaries:
        print("\n Cross-check summary ")
        print(f"Sum over layers: voxels (geometry) = {total_vox_geom}")
        if total_vox_energy > 0:
            print(f"Sum over layers: voxels (energy arrays) = {total_vox_energy}")
            print(f"Totals match: {total_vox_geom == total_vox_energy}")
        else:
            print("No energy_layer_* datasets found — skipped cross-check total.")
        # overall values:
        max_abs_x = max(abs(global_xmin), abs(global_xmax))
        max_abs_y = max(abs(global_ymin), abs(global_ymax))
        print(f"overall max|x| = {max_abs_x:.3f}, max|y| = {max_abs_y:.3f}\n")
        plt.tight_layout()
        plt.show()


In [1]:
path = "/fast_scratch_1/caloqvae/data/atlas_july31/eta_000/eta_000_default_binning/dataset_combined.hdf5"
#plot_all_layers_geometry_grid(path, layers=None, scale='equal_bin', cols=6)
# scale can be "equal_bin" or "linear"

In [5]:
# checking more geometry and binning:
def check_layer_consistency(h5file, layer, atol=1e-6, detailed=True):
    """
    For a given layer
      - groups voxels by (r0, r1) rings
      - counts phi bins per ring, sums delta_alpha to check if it wraps 2pi
      - confirms total voxels == sum n_phi(ring)
      - If n_phi is constant across rings, checks voxels == (#rings * n_phi)
    will return a dict with detailed stats
    """
    r0 = np.asarray(h5file[f'binstart_radius_layer_{layer}'])
    dr = np.asarray(h5file[f'binsize_radius_layer_{layer}'])
    a0 = np.asarray(h5file[f'binstart_alpha_layer_{layer}'])    # radians
    da = np.asarray(h5file[f'binsize_alpha_layer_{layer}'])     # radians
    r1 = r0 + dr

    n_vox = len(r0)
    if n_vox == 0:
        if detailed:
            print(f"[Layer {layer:2d}] EMPTY")
        return dict(layer=layer, empty=True)

    # identify rings by unique (r0, r1) pairs
    rings_pairs, inv = np.unique(np.stack([r0, r1], axis=1), axis=0, return_inverse=True)
    n_rings = rings_pairs.shape[0]

    # per-ring stats
    ring_stats = []
    phi_bins_per_ring = []
    coverage_ok = True

    for ring_id in range(n_rings):
        mask = (inv == ring_id)
        n_phi = int(mask.sum())
        phi_bins_per_ring.append(n_phi)

        # sort start angles in [0, 2pi) for a coverage check
        starts = np.sort(np.mod(a0[mask], 2*np.pi))
        widths = da[mask]
        # coverage by sum(widths)
        cov = widths.sum()

        ok = np.isclose(cov, 2*np.pi, atol=atol)
        if not ok:
            coverage_ok = False

        r0_ring, r1_ring = rings_pairs[ring_id]
        ring_stats.append(dict(
            ring_id=ring_id,
            r_in=float(r0_ring),
            r_out=float(r1_ring),
            n_phi=n_phi,
            phi_coverage=float(cov),
            wraps_2pi=bool(ok),
        ))

    # totals & consistency checks
    sum_over_rings = int(np.sum(phi_bins_per_ring))
    counts_match = (sum_over_rings == n_vox)

    # if all rings share same n_phi, check product
    unique_phi_bins = np.unique(phi_bins_per_ring)
    product_match = None
    if len(unique_phi_bins) == 1:
        product_match = (n_rings * unique_phi_bins[0] == n_vox)

    if detailed:
        print(f"\n[Layer {layer:2d}] voxels={n_vox}, rings={n_rings}, "
              f"phi-coverage all rings OK={coverage_ok}")
        print("per-ring (first 8 shown):")
        for s in ring_stats[:8]:
            print(f"ring {s['ring_id']:2d}: r in[{s['r_in']:.3f},{s['r_out']:.3f}], "
                  f"n_phi={s['n_phi']:3d}, coverage={s['phi_coverage']:.6f}, wraps={s['wraps_2pi']}")
        if len(ring_stats) > 8: # only print a certain amount, can change this to print more:
            print("...")

        print(f"sum n_phi(ring) = {sum_over_rings} | equals voxel count? (check) {counts_match}")
        if product_match is not None:
            print(f"(#rings * n_phi_per_ring) = {n_rings} * {unique_phi_bins[0]} = "
                  f"{n_rings * unique_phi_bins[0]} | equals voxel count? (check) {product_match}")
        else:
            print("issue with check")

    return dict(
        layer=layer,
        n_voxels=n_vox,
        n_rings=n_rings,
        phi_bins_per_ring=phi_bins_per_ring,
        sum_phi_bins=sum_over_rings,
        counts_match=counts_match,
        product_match=product_match,
        coverage_all_ok=coverage_ok,
        ring_stats=ring_stats,
    )

def check_all_layers(h5_path, layers=None, atol=1e-6):
    """
    Runs check_layer_consistency over all (or selected) layers and prints a summary
    """
    results = []
    with h5py.File(h5_path, "r") as f:
        if layers is None:
            layers = sorted({
                int(k.split('_')[-1])
                for k in f.keys()
                if k.startswith('binstart_radius_layer_')
            })
        total_geom = 0
        total_energy = 0
        all_ok = True

        for L in layers:
            res = check_layer_consistency(f, L, atol=atol, detailed=True)
            results.append(res)
            if not res.get('counts_match', True) or not res.get('coverage_all_ok', True):
                all_ok = False
            total_geom += res.get('n_voxels', 0)

            # cross-check against energy arrays
            if f.get(f'energy_layer_{L}') is not None:
                total_energy += f[f'energy_layer_{L}'].shape[1]

        print("\nLayered Totals")
        print(f"Total voxels (geometry): {total_geom}")
        if total_energy > 0:
            print(f"Total voxels (energy arrays): {total_energy} | match: {total_geom == total_energy}")
        else:
            print("Energy arrays not found -> skipped energy total check.")

        print(f"All layers passed (counts and phi-coverage): {all_ok}")
    return results


In [2]:
# path = "/fast_scratch_1/caloqvae/data/atlas_regular/dataset_eta_020_positive.hdf5"
path = "/fast_scratch_1/caloqvae/data/atlas_july31/eta_000/eta_000_regular_binning/dataset_combined.hdf5"

# check EVERY layer in the file
#_ = check_all_layers(path)

# choose specific layers to test
#_ = check_all_layers(path, layers=[0,1,2,3,12])


In [7]:
def _equal_bin_transform(r0, r1):
    """
    used to transform binning
    """
    bounds = np.unique(np.concatenate([r0, r1]))
    nb = len(bounds) - 1
    plot_bounds = np.linspace(0.0, 1.0, nb+1)
    def xform(R):
        idx = np.searchsorted(bounds, R, side='right') - 1
        idx = np.clip(idx, 0, nb-1)
        frac = (R - bounds[idx]) / (bounds[idx+1] - bounds[idx] + 1e-12)
        return plot_bounds[idx] + frac*(plot_bounds[idx+1] - plot_bounds[idx])
    return xform

def _ring_phi_summary(r0, a0):
    """
    summarize phi segmentation per ring.
    assumes r0 contains repeated inner-boundaries per ring and a0 is aligned with r0.
    returns a list of dicts per ring with counts and 2pi coverage check.
    """
    uniq_r, idx_start, counts = np.unique(r0, return_index=True, return_counts=True)
    out = []
    for k in range(len(uniq_r)):
        sl = slice(idx_start[k], idx_start[k] + counts[k])
        starts = np.sort(a0[sl])
        circular = np.concatenate([starts, [starts[0] + 2*np.pi]])
        dtheta = np.diff(circular)
        cov = dtheta.sum()
        out.append(dict(r_inner=float(uniq_r[k]), n_phi=int(counts[k]), phi_coverage=cov))
    return out

def summarize_layer_geometry_from_h5(h5file, layer, scale='equal_bin', title=None):
    # loading the geometry
    r0 = np.asarray(h5file[f'binstart_radius_layer_{layer}'])
    dr = np.asarray(h5file[f'binsize_radius_layer_{layer}'])
    a0 = np.asarray(h5file[f'binstart_alpha_layer_{layer}'])   # radians
    da = np.asarray(h5file[f'binsize_alpha_layer_{layer}'])    # radians

    r1  = r0 + dr
    a1  = a0 + da
    a0d = np.rad2deg(a0)
    a1d = np.rad2deg(a1)

    # geometry summary:
    voxels = len(r0)
    ring_summ = _ring_phi_summary(r0, a0)
    wraps_ok = all(abs(s['phi_coverage'] - 2*np.pi) < 1e-3 for s in ring_summ)

    # per layer physical x and y values:
    if r1.size:
        rmax = float(r1.max())
        ang_ext = np.concatenate([a0, a1, np.array([0.0, 0.5*np.pi, np.pi, 1.5*np.pi])])
        r_ext  = np.concatenate([r1, r1, np.full(4, rmax)])
        x = r_ext * np.cos(ang_ext)
        y = r_ext * np.sin(ang_ext)
        xmin, xmax = float(x.min()), float(x.max())
        ymin, ymax = float(y.min()), float(y.max())
    else:
        xmin = xmax = ymin = ymax = 0.0

    print(f"[Layer {layer:02d}] voxels={voxels}, rings={len(ring_summ)}, wraps_2pi={wraps_ok} | "
          f"x vals:[{xmin:.3f}, {xmax:.3f}], y vals:[{ymin:.3f}, {ymax:.3f}]")
    return r1, a0, a1, (xmin, xmax, ymin, ymax)

def summarize_all_layers_geometry(h5_path, layers=None, scale='equal_bin'):
    """
    function to summarize the results for the layers
    """
    with h5py.File(h5_path, "r") as f:
        # get the layers:
        if layers is None:
            layers = sorted({
                int(k.split('_')[-1])
                for k in f.keys()
                if k.startswith('binstart_radius_layer_')
            })

        global_xmax = 0.0
        global_ymax = 0.0
        per_layer = {}
        
        # loop through the layers
        for L in layers:
            r1, a0, a1, (xmin, xmax, ymin, ymax) = summarize_layer_geometry_from_h5(
                f, layer=L, scale=scale, title=f'Layer {L}')
            global_xmax = max(global_xmax, abs(xmin), abs(xmax))
            global_ymax = max(global_ymax, abs(ymin), abs(ymax))
            per_layer[L] = {
                "r_outer": r1, "alpha_start": a0, "alpha_end": a1,
                "extents": dict(xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax),
            }
        # print overall results:
        print(f"global max for x and y: max|x| = {global_xmax:.3f}, max|y| = {global_ymax:.3f}\n")

In [3]:
path = "/fast_scratch_1/caloqvae/data/atlas_july31/eta_000/eta_000_default_binning/dataset_combined.hdf5"
#summarize_all_layers_geometry(path)

In [9]:
def plot_layer_geometry_from_h5_2(h5file, layer, scale='equal_bin', ax=None, title=None,
                                edgecolor='k', linewidth=0.4):
    """
    prints and return values
    """
    # loading the geometry
    r0 = np.asarray(h5file[f'binstart_radius_layer_{layer}'])
    dr = np.asarray(h5file[f'binsize_radius_layer_{layer}'])
    a0 = np.asarray(h5file[f'binstart_alpha_layer_{layer}'])   # radians
    da = np.asarray(h5file[f'binsize_alpha_layer_{layer}'])    # radians

    r1  = r0 + dr
    a1  = a0 + da
    a0d = np.rad2deg(a0)
    a1d = np.rad2deg(a1)
    _ = (a0d, a1d) 

    # prints:
    voxels = len(r0)
    ring_summ = _ring_phi_summary(r0, a0)
    wraps_ok = all(abs(s['phi_coverage'] - 2*np.pi) < 1e-3 for s in ring_summ)

    print(f"[Layer {layer:02d}] voxels={voxels}, rings={len(ring_summ)}, wraps_2pi={wraps_ok}")
    for s in ring_summ[:5]:
        print(f"  ring r_in={s['r_inner']:.3f}: n_phi={s['n_phi']}, coverage={s['phi_coverage']:.6f} rad")
    if len(ring_summ) > 5:
        print("...") # just check the first 5 rings but can extend this to print them all

    # equal_bin scale (can also do linear)
    if scale == 'equal_bin' and r1.size:
        _ = _equal_bin_transform(r0, r1)

    return r1, a0, a1

def plot_all_layers_geometry_grid(h5_path, layers=None, scale='equal_bin', cols=6, figsize=(18, 12)):
    """
    iterates layers with the same per-layer prints
    """
    with h5py.File(h5_path, "r") as f:
        # discover layers
        if layers is None:
            layers = sorted({
                int(k.split('_')[-1])
                for k in f.keys()
                if k.startswith('binstart_radius_layer_')
            })

        global_xmax = 0.0
        global_ymax = 0.0

        for L in layers:
            r1, a0, a1 = plot_layer_geometry_from_h5_2(
                f, layer=L, scale=scale, ax=None, title=f'Layer {L}')

            # compute global physical max x and y values
            ang = np.concatenate([a0, a1])
            r   = np.concatenate([r1, r1])
            x = r * np.cos(ang)
            y = r * np.sin(ang)
            if x.size:
                global_xmax = max(global_xmax, float(np.max(np.abs(x))))
                global_ymax = max(global_ymax, float(np.max(np.abs(y))))

        print(f"\nglobal: max|x| = {global_xmax:.3f}, max|y| = {global_ymax:.3f}\n")

In [4]:
# using the previous function:
path = "/fast_scratch_1/caloqvae/data/atlas_july31/eta_000/eta_000_regular_binning/dataset_combined.hdf5"
#plot_all_layers_geometry_grid(path, scale='equal_bin')

In [11]:
def plot_all_layers_subplot(raw_file, event_n, all_layers=None, scale='equal_bin', vmin=1e-2, vmax=1e4):
    if all_layers is None:
        all_layers = [i for i in range(24)]  # defaults to all layers
    
    atlas_plotter = PLT_ATLAS(raw_file, event_n, all_layers)
    
    num_layers = len(all_layers)
    cols = 6  # number of subplots per row
    rows = int(np.ceil(num_layers / cols))

    fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows), subplot_kw={'aspect': 'equal'})
    axes = axes.flatten()

    # global normalization
    norm = LogNorm(vmin=vmin, vmax=vmax)

    incident_energy = torch.tensor(atlas_plotter.data["incident_energy"][event_n])
    print(f"event info: event_n = {event_n}, incident_energy = {incident_energy.item()}")

    for i, layer in enumerate(all_layers):
        # load layer-specific data
        energy_info = atlas_plotter.data[f'energy_layer_{layer}'][event_n, :]
        binsize_alpha = atlas_plotter.data[f"binsize_alpha_layer_{layer}"]
        binstart_alpha = atlas_plotter.data[f"binstart_alpha_layer_{layer}"]
        binsize_radius = atlas_plotter.data[f"binsize_radius_layer_{layer}"]
        binstart_radius = atlas_plotter.data[f"binstart_radius_layer_{layer}"]
        single_event_energy = energy_info * incident_energy

        atlas_plotter.binsize_alpha_info = binsize_alpha
        atlas_plotter.binstart_alpha_info = binstart_alpha
        atlas_plotter.binsize_radius_info = binsize_radius
        atlas_plotter.binstart_radius_info = binstart_radius
        atlas_plotter.single_event_energy = single_event_energy

        atlas_plotter.sectors = atlas_plotter.get_section_info()

        atlas_plotter.plot_calorimeter(
            ax=axes[i], scale=scale, norm=norm,
            title=f'Layer {layer}'
        )

    # remove axes not being used
    for j in range(i+1, len(axes)):
        axes[j].axis('off')

    fig.suptitle(f'Calorimeter Layers – Scale: {scale}, Event E = {incident_energy/1000:.2f} GeV', fontsize=18)

    # adding a shared colorbar
    cbar_ax = fig.add_axes([0.25, 0.05, 0.5, 0.02])
    cmap = 'rainbow'
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal')
    cbar.set_label('Energy in MeV', fontsize=14)
    cbar.ax.tick_params(labelsize=12)

    plt.tight_layout(rect=[0, 0.07, 1, 0.95])
    plt.show()


In [5]:
raw_file = path = "/fast_scratch_1/caloqvae/data/atlas_july31/eta_000/eta_000_regular_binning/dataset_combined.hdf5"
event_n = 0  # choosing a random event index (here choosing 0)

# plot all layers in a grid, using equal-bin scale
# plot_all_layers_subplot(
#     raw_file,
#     event_n,
#     all_layers=list(range(24)),
#     scale='equal_bin', # or could do 'linear'
#     vmin=1e-2, vmax=1e4
# )

In [13]:
# plotting all cylinder layers:

In [6]:
path = "/fast_scratch_1/caloqvae/data/atlas_regular/dataset_eta_020_positive.hdf5"
#path = "/fast_scratch_1/caloqvae/data/atlas_july31/eta_000/eta_000_regular_binning/dataset_combined.hdf5"
#path = '/fast_scratch_1/caloqvae/data/atlas_july31/eta_130/eta_130_regular_binning/19/dataset_combined.hdf5'

# open file and plot
# with h5py.File(path, "r") as f:
#     draw_cylinder(f)

In [7]:
path = "/fast_scratch_1/caloqvae/data/atlas_july31/eta_000/eta_000_regular_binning/dataset_combined.hdf5"
#path = '/fast_scratch_1/caloqvae/data/atlas_july31/eta_130/eta_130_regular_binning/19/dataset_combined.hdf5'

# open file and plot
# with h5py.File(path, "r") as f:
#     draw_cylinder(f)

In [12]:
import matplotlib as mpl


def draw_cylinder_selected_layers(h5_path, layers=(0,1,2,3,12), layer_thickness=1.0, num_points=24, cmap_name="viridis"):
    """
    draw the 3D calorimeter cylinder geometry for a selected set of layers
    geometry only (no energies), uses binstart_radius_layer_* and binstart_alpha_layer_*
    """
    cmap = mpl.colormaps[cmap_name]

    # read the geometry
    radii_cache = {}
    theta_cache = {}
    global_max_r = 0.0

    # open and read the file
    with h5py.File(h5_path, "r") as f:
        for L in layers:
            r_starts = np.asarray(f[f"binstart_radius_layer_{L}"])  # inner ring boundaries
            a_starts = np.asarray(f[f"binstart_alpha_layer_{L}"])   # angular boundaries (radians)
            radii_cache[L] = r_starts
            theta_cache[L] = a_starts
            if r_starts.size:
                global_max_r = max(global_max_r, float(r_starts.max()))

    # plotting
    fig = plt.figure(figsize=(10, 8))
    ax  = fig.add_subplot(111, projection="3d")

    for i, L in enumerate(layers):
        z_low  = i * layer_thickness
        z_high = (i + 1) * layer_thickness
        color  = cmap(i / max(1, len(layers)-1))
        add_layer_varying(
            ax,
            radii_cache[L], # repeated ring radii (inner boundaries)
            theta_cache[L], # theta boundaries (radians)
            z_low, z_high,
            num_points=num_points,
            color=color
        )

    ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
    ax.set_xlim(-global_max_r, global_max_r)
    ax.set_ylim(-global_max_r, global_max_r)
    ax.set_zlim(0, len(layers) * layer_thickness)
    # visual help:
    ax.set_box_aspect([1, 1, (len(layers)*layer_thickness)/(2*global_max_r + 1e-9)])
    plt.title(f"Calorimeter cylinder (layers {list(layers)})")
    plt.tight_layout()
    plt.show()

In [12]:
# plotting selected layers:

# path = "/fast_scratch_1/caloqvae/data/atlas_regular/dataset_eta_020_positive.hdf5"
path = "/fast_scratch_1/caloqvae/data/atlas_july31/eta_000/eta_000_default_binning/dataset_combined.hdf5"

# draw_cylinder_selected_layers(
#     h5_path=path,
#     layers=(0,1,2,3,12),
#     layer_thickness=180.0,  
#     num_points=48,
#     cmap_name="viridis"
# )

In [16]:
def get_active_layers(plt_atlas, event_n, threshold=0.0):
    """
    return layers where the event has any energy above `threshold`
    Uses the tensors already loaded in plt_atlas.data
    """
    active = []
    for L in range(24):
        e = plt_atlas.data[f'energy_layer_{L}'][event_n].numpy()
        if np.any(e):
            active.append(L)
    return active

In [13]:
path = "/fast_scratch_1/caloqvae/data/atlas_july31/eta_000/eta_000_default_binning/dataset_combined_fine.hdf5"

with h5py.File(path, "r") as f:
    incident = f["incident_energy"][:]  # shape (N_events,)
    max_idx = np.argmax(incident)
    max_val = incident[max_idx]

#print(f"Event with highest incident energy: {max_idx}")
#print(f"Incident energy value: {max_val} MeV")

In [14]:
#path = '/fast_scratch_1/caloqvae/data/atlas_july31/eta_130/eta_130_regular_binning/0/dataset_combined.hdf5'
path = "/fast_scratch_1/caloqvae/data/atlas_july31/eta_000/eta_000_regular_binning/dataset_combined_fine.hdf5"

with h5py.File(path, "r") as f:
    incident = f["incident_energy"][:]  # shape (N_events,)
    max_idx = np.argmax(incident)
    max_val = incident[max_idx]

#print(f"Event with highest incident energy: {max_idx}")
#print(f"Incident energy value: {max_val} MeV")

In [17]:
path = "/fast_scratch_1/caloqvae/data/atlas_july31/eta_000/eta_000_default_binning/dataset_combined_fine.hdf5"
#path = '/fast_scratch_1/caloqvae/data/atlas_july31/eta_130/eta_130_regular_binning/0/dataset_combined.hdf5'

# pick a given event index
#event_n = 19709
event_n = 19709

# getting active layers
plt_atlas = PLT_ATLAS(raw_dir=path, event_n=event_n)
active = get_active_layers(plt_atlas, event_n, threshold=0.0)

# limit plotting to active layers
plt_atlas.all_layers = active
#print("Active layers:", active)

# plotting
#plt_atlas.plot_all_layers(event_n=event_n, scale="equal_bin", vmin=1e-2, vmax=1e4)